In [ ]:
import sys
from importlib import reload

import polars as pl

sys.path.append("..")
from src import util


In [4]:
data = util.load_examples_json()
for r in data.filter(pl.col("tokens").list.contains("<")).iter_rows(named=True):
    print("".join(r["tokens"]))
    print(r["tags"], "\n\n")


Loaded 198 examples
for (i = 0; i < 1000; i++)
    {
        if (states[i] != 1)
            res = 0;
    }
['kwfl', 'ws', 'brop', 'va', 'ws', 'opas', 'ws', 'nu', 'pu', 'ws', 'va', 'ws', 'opcm', 'ws', 'nu', 'pu', 'ws', 'va', 'opun', 'brcl', 'nl', 'id', 'brop', 'nl', 'id', 'id', 'kwfl', 'ws', 'brop', 'va', 'brop', 'va', 'brcl', 'ws', 'opcm', 'ws', 'nu', 'brcl', 'nl', 'id', 'id', 'id', 'va', 'ws', 'opas', 'ws', 'nu', 'pu', 'nl', 'id', 'brcl'] 


fn gen_row_set_par(n: &BigUint, jmax: usize, nrows: usize, fb: &[usize]) -> HashSet<Row>
['kwfn', 'ws', 'fnfr', 'brop', 'pa', 'sy', 'ws', 'opun', 'cl', 'pu', 'ws', 'pa', 'sy', 'ws', 'kwty', 'pu', 'ws', 'pa', 'sy', 'ws', 'kwty', 'pu', 'ws', 'pa', 'sy', 'ws', 'opun', 'brop', 'kwty', 'brcl', 'brcl', 'ws', 'sy', 'ws', 'cl', 'sy', 'cl', 'sy'] 


pub fn sum_primint<T>(a: &Vec<T>) -> T
where
    T: PrimInt,
{
    let mut s = T::zero();

    for x in a {
        s = s + *x;
    }

    return s;
}
['kwmo', 'ws', 'kwfn', 'ws', 'fnfr', 'sy', 'cl', 'sy', 'br

In [6]:
brop = ["(", "[", "{"]
brcl = [")", "]", "}"]


def repl(token: str, tag: str):
    if tag != "br":
        return tag
    if token in brop:
        return "brop"
    elif token in brcl:
        return "brcl"


def repl_row(row):
    tags_new = []
    for token, tag in zip(*row):
        tags_new.append(repl(token, tag))
    return tags_new


tags_new = []
for r in data.select("tokens", "tags").iter_rows():
    tags_new.append(repl_row(r))

data = data.with_columns(pl.Series("tags", tags_new))
data

difficulty,tokens,tags,name,lang,length
str,list[str],list[str],str,str,u32
"""easy""","[""x"", ""="", ""1""]","[""va"", ""opas"", ""nu""]","""shrt_pseudo""","""pseudo""",3
"""easy""","[""["", ""2"", … ""]""]","[""brop"", ""nu"", … ""brcl""]","""smplrr_json""","""json""",9
"""normal""","[""say"", "" "", """"Hello world""""]","[""kwio"", ""ws"", ""st""]","""hllwrld_natural""","""natural""",3
"""normal""","[""puts"", "" "", """"Hello World""""]","[""kwio"", ""ws"", ""st""]","""hllwrld_ruby""","""ruby""",3
"""normal""","[""if"", "" "", … "")""]","[""kwfl"", ""ws"", … ""brcl""]","""smplndnttb_pseudo""","""pseudo""",8
…,…,…,…,…,…
"""ambiguous""","[""df"", ""."", … "")""]","[""va"", ""sy"", … ""brcl""]","""plwhnthn_python""","""python""",45
"""normal""","[""$user"", "" "", … ""}""]","[""va"", ""id"", … ""brcl""]","""fndsr_php""","""php""",42
"""normal""","[""class"", "" "", … ""}""]","[""kwde"", ""ws"", … ""brcl""]","""crdtcnst_php""","""php""",31


In [7]:
util.save_examples_json(data, "../data/newexamples.json")
